In [ ]:
# GBA 462 Lab 4

# Liuyi Ye / Xinyi Wang

# 10/04/2023

Case Study: CAPM (Similar to HW3 Q1)

1. Estimating β of 4 individual firm stocks (Microsoft, IBM, Walmart, Target) for the calendar years
1990 through 1999 using the Capital Asset Pricing Model (CAPM):

$$ R - R_f = \alpha + \beta (R_m - R_f) $$

where R is the expected return of the firm’s stock, Rm is the expected return of the market, and Rf is the
risk free rate.

Note:
1. α (the intercept) is assumed to be 0 in theory, but we will include it in the estimation here.
2. The tickers of the 4 individual stocks are MSFT, IBM, WMT, TGT, respectively. Collect data on
stock prices from either Yahoo finance or Google Finance and then choose a data frequency. As an
example, we choose a monthly frequency.
3. We will obtain the market expected excess returns $R_m − R_f$ and the risk free rate $R_f$ from the
Fama-French website (Alternatively, you can choose other appropriate market indices/market returns
and risk free rates and collect data from other sources).

In [ ]:
## import packages
import pandas as pd
import numpy as np
import scipy as sp
from scipy import optimize
import statsmodels.formula.api as smf
from datetime import datetime
import yfinance as yf


In [ ]:
microsoft = yf.download("MSFT", start= datetime(1990,1,1), end = datetime(2000,2,1), interval = '1mo')
ibm = yf.download("IBM", start= datetime(1990,1,1), end = datetime(2000,2,1), interval = '1mo')
walmart = yf.download("WMT", start= datetime(1990,1,1), end = datetime(2000,2,1), interval = '1mo')
target = yf.download("TGT", start= datetime(1990,1,1), end = datetime(2000,2,1), interval = '1mo')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
rf_mkt_names = ['date', 'mkt_rf', 'smb', 'hml', 'rf']
rf_mkt = pd.read_csv("F-F_Research_Data_Factors.CSV", skiprows = 4, nrows = 1158-4, \
names = rf_mkt_names, dtype = {'date': np.int32, 'mkt_rf': np.float64, \
'smb': np.float64, 'hml': np.float64, 'rf': np.float64})

In [ ]:
microsoft

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1990-01-01,0.605903,0.666667,0.583333,0.642361,0.399049,2380968000
1990-02-01,0.642361,0.697917,0.640625,0.685764,0.426012,1429344000
1990-03-01,0.684028,0.812500,0.684028,0.769097,0.477780,1717819200
1990-04-01,0.762153,0.875000,0.758681,0.805556,0.500429,1200837600
1990-05-01,0.815972,1.093750,0.788194,1.013889,0.629850,1634767200
...,...,...,...,...,...,...
1999-09-01,46.156250,48.937500,44.406250,45.281250,28.129711,977273600
1999-10-01,45.093750,47.625000,42.531250,46.281250,28.750938,1062304400
1999-11-01,46.625000,47.250000,42.187500,45.523438,28.280172,1465256000


In [ ]:
microsoft['lag_price'] = microsoft['Adj Close'].shift(1)


In [ ]:
microsoft

,Open,High,Low,Close,Adj Close,Volume,lag_price
Date,,,,,,,
1990-01-01,0.605903,0.666667,0.583333,0.642361,0.399049,2380968000,NaN
1990-02-01,0.642361,0.697917,0.640625,0.685764,0.426012,1429344000,0.399049
1990-03-01,0.684028,0.812500,0.684028,0.769097,0.477780,1717819200,0.426012
1990-04-01,0.762153,0.875000,0.758681,0.805556,0.500429,1200837600,0.477780
1990-05-01,0.815972,1.093750,0.788194,1.013889,0.629850,1634767200,0.500429
...,...,...,...,...,...,...,...
1999-09-01,46.156250,48.937500,44.406250,45.281250,28.129711,977273600,28.750938
1999-10-01,45.093750,47.625000,42.531250,46.281250,28.750938,1062304400,28.129711
1999-11-01,46.625000,47.250000,42.187500,45.523438,28.280172,1465256000,28.750938


In [ ]:
rf_mkt

,date,mkt_rf,smb,hml,rf
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31
...,...,...,...,...,...
1149,202204,-9.46,-1.41,6.19,0.01
1150,202205,-0.34,-1.85,8.41,0.03
1151,202206,-8.43,2.09,-5.97,0.06
1152,202207,9.57,2.81,-4.10,0.08


In [ ]:
def clean_stock(stock_df, rf_mkt_df):
### stock_df is the data frame containing individual stocks
### rf_mkt_df contains risk-free return and market return
### first drop NaN (API returns NaN, not relevant)
  new_stock_df = stock_df.dropna().copy()
  ### generate a column called 'date' with dtype = np.int32
  new_stock_df['date'] = new_stock_df.index
  new_stock_df['date'] = new_stock_df['date'].astype(str)
  new_stock_df['date'] = new_stock_df['date'].str[0:4] + new_stock_df['date'].str[5:7]
  new_stock_df['date'] = new_stock_df['date'].astype(np.int32)
  new_stock_df['lag_price'] = new_stock_df['Adj Close'].shift(1)

  new_stock_df['return'] = 100 * \
  (new_stock_df['Adj Close'] - new_stock_df['lag_price']  ) / new_stock_df['lag_price']
  # A different measure -> different estimate
  # new_stock_df['lead_price'] = new_stock_df['Adj Close'].shift(-1)
  # new_stock_df['return'] = 100 * \
  # (new_stock_df['lead_price'] - new_stock_df['Adj Close']) / new_stock_df['Adj Close']
  new_stock_df = new_stock_df.dropna()
  ### merge with rf_mkt_df
  output_df = new_stock_df.merge(rf_mkt_df, how = 'inner', on = 'date')
  output_df['excess_return'] = output_df['return'] - output_df['rf']
  output_df = output_df[['date', 'excess_return', 'rf', 'mkt_rf']]
  return output_df


In [ ]:
microsoft_reg = clean_stock(microsoft, rf_mkt)
ibm_reg = clean_stock(ibm, rf_mkt)
walmart_reg = clean_stock(walmart, rf_mkt)
target_reg = clean_stock(target, rf_mkt)


In [ ]:
microsoft_reg

,date,excess_return,rf,mkt_rf
0,199003,11.511795,0.64,1.83
1,199004,4.050495,0.69,-3.36
2,199005,25.182025,0.68,8.42
3,199006,3.479638,0.63,-1.09
4,199007,-13.180067,0.68,-1.90
...,...,...,...,...
114,199909,-2.550720,0.39,-2.79
115,199910,1.818438,0.39,6.12
116,199911,-1.997394,0.36,3.37
117,199912,27.790604,0.44,7.72


Regression

In [ ]:
microsoft_est = smf.ols(formula = 'excess_return ~ mkt_rf', data = microsoft_reg).fit()
print(microsoft_est.summary())

                            OLS Regression Results                            
Dep. Variable:          excess_return   R-squared:                       0.303
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     50.96
Date:                Wed, 04 Oct 2023   Prob (F-statistic):           8.54e-11
Time:                        17:59:26   Log-Likelihood:                -417.42
No. Observations:                 119   AIC:                             838.8
Df Residuals:                     117   BIC:                             844.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.2231      0.774      2.870      0.0

In [ ]:
ibm_est = smf.ols(formula = 'excess_return ~ mkt_rf', data = ibm_reg).fit()
print(ibm_est.summary())

                            OLS Regression Results                            
Dep. Variable:          excess_return   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.136
Method:                 Least Squares   F-statistic:                     19.67
Date:                Wed, 04 Oct 2023   Prob (F-statistic):           2.08e-05
Time:                        17:59:28   Log-Likelihood:                -416.88
No. Observations:                 120   AIC:                             837.8
Df Residuals:                     118   BIC:                             843.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5446      0.746      0.730      0.4

In [ ]:
walmart_est = smf.ols(formula = 'excess_return ~ mkt_rf', data = walmart_reg).fit()
print(walmart_est.summary())

                            OLS Regression Results                            
Dep. Variable:          excess_return   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     51.42
Date:                Wed, 04 Oct 2023   Prob (F-statistic):           7.05e-11
Time:                        17:59:36   Log-Likelihood:                -394.57
No. Observations:                 120   AIC:                             793.1
Df Residuals:                     118   BIC:                             798.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7193      0.619      1.162      0.2

In [ ]:
target_est = smf.ols(formula = 'excess_return ~ mkt_rf', data = target_reg).fit()
print(target_est.summary())


                            OLS Regression Results                            
Dep. Variable:          excess_return   R-squared:                       0.374
Model:                            OLS   Adj. R-squared:                  0.369
Method:                 Least Squares   F-statistic:                     70.53
Date:                Wed, 04 Oct 2023   Prob (F-statistic):           1.16e-13
Time:                        17:59:40   Log-Likelihood:                -391.41
No. Observations:                 120   AIC:                             786.8
Df Residuals:                     118   BIC:                             792.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3051      0.603      0.506      0.6

Case Study: Demand Analysis (Similar to HW3 Q2)

The price and quantity data of three plant-based milk brands (Brand 1, Brand 2, and Brand 3) is compiled
in the file lab_data.xlsx. For example, $q_1$ is the quantity (in ounces) of Brand 1 plant-based milk sold, and $p_1$ is the price (in $) of Brand 1 plant-based milk.

a. For each of the three brands (1, 2, and 3) in the data, run regressions of the following type.

$$q_t = α + β p_t + u_t$$


b. After obtaining the regression results (i.e., the estimated demand equation), compute each brand’s own
price elasticity at (average p, average q).

• Note: Price elasticity $$\epsilon  = \frac{\partial{q}}{\partial{p}}\frac{p}{q}$$


c. Assume that the cost of producing an additional unit is 1 cent per ounce for each of the three brands.
Compute the optimal price for each brand based on the estimated demand equation.

In [ ]:
pbm = pd.read_excel('lab_data.xlsx', sheet_name = 'pbm')
est1 = smf.ols(formula='q1 ~ p1', data=pbm).fit()
print(est1.summary())


                            OLS Regression Results                            
Dep. Variable:                     q1   R-squared:                       0.564
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     279.4
Date:                Wed, 04 Oct 2023   Prob (F-statistic):           8.43e-41
Time:                        18:04:08   Log-Likelihood:                -3389.0
No. Observations:                 218   AIC:                             6782.
Df Residuals:                     216   BIC:                             6789.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.681e+07   7.82e+05     21.503      0.0

In [ ]:
elasticity = est1.params['p1'] * pbm['p1'].mean() / pbm['q1'].mean()
print(elasticity)


-3.382005585163894


In [ ]:
est2 = smf.ols(formula='q2 ~ p2', data=pbm).fit()
print(est2.summary())


                            OLS Regression Results                            
Dep. Variable:                     q2   R-squared:                       0.403
Model:                            OLS   Adj. R-squared:                  0.400
Method:                 Least Squares   F-statistic:                     145.7
Date:                Wed, 04 Oct 2023   Prob (F-statistic):           5.62e-26
Time:                        18:06:13   Log-Likelihood:                -3432.2
No. Observations:                 218   AIC:                             6868.
Df Residuals:                     216   BIC:                             6875.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.217e+07   8.11e+05     15.000      0.0

In [ ]:
elasticity = est2.params['p2'] * pbm['p2'].mean() / pbm['q2'].mean()
print(elasticity)


-3.9208368506774907


In [ ]:
est3 = smf.ols(formula='q3 ~ p3', data=pbm).fit()
print(est3.summary())

                            OLS Regression Results                            
Dep. Variable:                     q3   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.364
Method:                 Least Squares   F-statistic:                     125.0
Date:                Wed, 04 Oct 2023   Prob (F-statistic):           3.38e-23
Time:                        18:06:19   Log-Likelihood:                -3441.5
No. Observations:                 218   AIC:                             6887.
Df Residuals:                     216   BIC:                             6894.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.213e+07   8.57e+05     14.157      0.0

In [ ]:
elasticity = est3.params['p3'] * pbm['p3'].mean() / pbm['q3'].mean()
print(elasticity)

-3.592538202831608


Finding optimal price

Approach 1: Take derivative with respect to p and set it to 0

$$Profit = (p-c) \times Q = (p-0.01)(\alpha + \beta p ) = \alpha p +\beta p^2 - 0.01\alpha \times \beta p $$

F.O.C $$\Rightarrow  \alpha + 2\beta p - 0.01\alpha\beta = 0  \rightarrow p = \frac{0.01\beta - \alpha}{2\beta}$$

Approach 2: use optimizer

In [ ]:
# we minimize -Q
# this is the default. You can also maximize Q

def f1(p):
  return -(p - .01) * (est1.params['Intercept'] + est1.params['p1'] * p)


result1 = optimize.minimize_scalar(f1)
print(result1.success) # check if solver was successful


True


In [ ]:
result1

 message: 
          Optimization terminated successfully;
          The returned value satisfies the termination criteria
          (using xtol = 1.48e-08 )
 success: True
     fun: -388476.0150462232
       x: 0.060775315087482205
     nit: 5
    nfev: 8

In [ ]:
x_min1 = result1.x
x_min1

0.060775315087482205

In [ ]:
def f2(x):
  return -(x - .01) * (est2.params['Intercept'] + est2.params['p2'] * x)
result2 = optimize.minimize_scalar(f2)
print(result2.success) # check if solver was successful

True


In [ ]:
x_min2 = result2.x
x_min2

0.07232747758980446

In [ ]:
def f3(x):
  return -(x - .01) * (est3.params['Intercept'] + est3.params['p3'] * x)
result3 = optimize.minimize_scalar(f3)
print(result3.success) # check if solver was successful

True


In [ ]:
x_min3 = result3.x
x_min3

0.0731775355983832